In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np

In [16]:
# 1. Dataset Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [17]:
# Memastikan penggunaan GPU jika tersedia
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device Name:", torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

CUDA Available: True
GPU Device Name: NVIDIA GeForce GTX 1660
Using device: cuda


In [18]:
# 2. CNN Model
class CNNModel(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNNModel, self).__init__()
        pool_layer = nn.MaxPool2d if pooling_type == 'max' else nn.AvgPool2d

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=kernel_size, stride=1, padding=kernel_size // 2),
            nn.ReLU(),
            pool_layer(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=kernel_size, stride=1, padding=kernel_size // 2),
            nn.ReLU(),
            pool_layer(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [19]:
# 3. Training and Testing Functions
def train_model(model, optimizer, criterion, train_loader, device):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    return train_loss / len(train_loader)

def test_model(model, criterion, test_loader, device):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = correct / len(test_loader.dataset)
    return test_loss / len(test_loader), accuracy

In [20]:
# 4. Experiment Configurations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100, 250, 350]
optimizers = {
    "SGD": lambda params: optim.SGD(params, lr=0.01, momentum=0.9),
    "RMSProp": lambda params: optim.RMSprop(params, lr=0.001),
    "Adam": lambda params: optim.Adam(params, lr=0.001)
}

In [21]:
# 5. Running Experiments
results = []

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer_name, optimizer_func in optimizers.items():
            for num_epochs in epochs_list:
                print(f"Kernel Size: {kernel_size}, Pooling: {pooling_type}, Optimizer: {optimizer_name}, Epochs: {num_epochs}")

                # Model and optimizer setup
                model = CNNModel(kernel_size=kernel_size, pooling_type=pooling_type).to(device)
                criterion = nn.CrossEntropyLoss()
                optimizer = optimizer_func(model.parameters())
                scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Learning rate scheduler

                best_accuracy = 0
                patience, patience_counter = 10, 0  # Early stopping config

                for epoch in range(num_epochs):
                    train_loss = train_model(model, optimizer, criterion, train_loader, device)
                    test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
                    scheduler.step()

                    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

                    # Early stopping logic
                    if test_accuracy > best_accuracy:
                        best_accuracy = test_accuracy
                        patience_counter = 0
                    else:
                        patience_counter += 1
                        if patience_counter >= patience:
                            print("Early stopping triggered!")
                            break

                # Save results
                results.append({
                    "Kernel Size": kernel_size,
                    "Pooling": pooling_type,
                    "Optimizer": optimizer_name,
                    "Epochs": num_epochs,
                    "Best Accuracy": best_accuracy
                })

Kernel Size: 3, Pooling: max, Optimizer: SGD, Epochs: 5
Epoch 1/5 - Train Loss: 1.9483, Test Loss: 1.5929, Test Accuracy: 0.4305
Epoch 2/5 - Train Loss: 1.5308, Test Loss: 1.3546, Test Accuracy: 0.5109
Epoch 3/5 - Train Loss: 1.3701, Test Loss: 1.2666, Test Accuracy: 0.5413
Epoch 4/5 - Train Loss: 1.2732, Test Loss: 1.1770, Test Accuracy: 0.5798
Epoch 5/5 - Train Loss: 1.1876, Test Loss: 1.1059, Test Accuracy: 0.6031
Kernel Size: 3, Pooling: max, Optimizer: SGD, Epochs: 50
Epoch 1/50 - Train Loss: 1.9406, Test Loss: 1.5807, Test Accuracy: 0.4312
Epoch 2/50 - Train Loss: 1.5079, Test Loss: 1.3477, Test Accuracy: 0.5185
Epoch 3/50 - Train Loss: 1.3518, Test Loss: 1.2355, Test Accuracy: 0.5605
Epoch 4/50 - Train Loss: 1.2539, Test Loss: 1.1380, Test Accuracy: 0.5929
Epoch 5/50 - Train Loss: 1.1622, Test Loss: 1.0743, Test Accuracy: 0.6173
Epoch 6/50 - Train Loss: 1.0833, Test Loss: 1.0222, Test Accuracy: 0.6335
Epoch 7/50 - Train Loss: 1.0171, Test Loss: 0.9597, Test Accuracy: 0.6596
Epoc

In [22]:
# 6. Display Results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

# Optionally save the results
results_df.to_csv("cifar10_cnn_experiments.csv", index=False)

    Kernel Size Pooling Optimizer  Epochs  Best Accuracy
0             3     max       SGD       5         0.6031
1             3     max       SGD      50         0.7246
2             3     max       SGD     100         0.7199
3             3     max       SGD     250         0.7125
4             3     max       SGD     350         0.7258
..          ...     ...       ...     ...            ...
85            7     avg      Adam       5         0.6710
86            7     avg      Adam      50         0.7533
87            7     avg      Adam     100         0.7367
88            7     avg      Adam     250         0.7415
89            7     avg      Adam     350         0.7377

[90 rows x 5 columns]
